In [162]:
import pandas as pd
import numpy as np

In [163]:
train_df = pd.read_csv("train.csv")

In [164]:
train_df.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X376,X377,X378,X379,X380,X382,X383,X384,X385,y
0,0,k,v,at,a,d,u,j,o,0,...,0,1,0,0,0,0,0,0,0,130.81
1,6,k,t,av,e,d,y,l,o,0,...,0,0,0,0,0,0,0,0,0,88.53
2,7,az,w,n,c,d,x,j,x,0,...,0,0,0,0,0,1,0,0,0,76.26
3,9,az,t,n,f,d,x,l,e,0,...,0,0,0,0,0,0,0,0,0,80.62
4,13,az,v,n,f,d,h,d,n,0,...,0,0,0,0,0,0,0,0,0,78.02


In [165]:
train_df.shape

(4209, 378)

In [166]:
test_df = pd.read_csv("test.csv")

### Preprocessing

In [167]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4209 entries, 0 to 4208
Columns: 378 entries, ID to y
dtypes: float64(1), int64(369), object(8)
memory usage: 12.1+ MB


In [168]:
cat_variables = train_df.select_dtypes("O").columns

In [169]:
cat_variables

Index(['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8'], dtype='object')

In [170]:
train_df[cat_variables]

,X0,X1,X2,X3,X4,X5,X6,X8
0,k,v,at,a,d,u,j,o
1,k,t,av,e,d,y,l,o
2,az,w,n,c,d,x,j,x
3,az,t,n,f,d,x,l,e
4,az,v,n,f,d,h,d,n
...,...,...,...,...,...,...,...,...
4204,ak,s,as,c,d,aa,d,q
4205,j,o,t,d,d,aa,h,h
4206,ak,v,r,a,d,aa,g,e
4207,al,r,e,f,d,aa,l,u


In [171]:
print(train_df[cat_variables].nunique())

X0    47
X1    27
X2    44
X3     7
X4     4
X5    29
X6    12
X8    25
dtype: int64


In [172]:
# Removing duplicated columns
def get_duplicate_columns(df):

    duplicate_columns = {}
    seen_columns = {}

    for column in df.columns:
        current_column = df[column]

        # Convert column data to bytes
        try:
            current_column_hash = current_column.values.tobytes()
        except AttributeError:
            current_column_hash = current_column.to_string().encode()

        if current_column_hash in seen_columns:
            if seen_columns[current_column_hash] in duplicate_columns:
                duplicate_columns[seen_columns[current_column_hash]].append(
                    column)
            else:
                duplicate_columns[seen_columns[current_column_hash]] = [column]
        else:
            seen_columns[current_column_hash] = column

    return duplicate_columns

In [173]:
duplicate_columns = get_duplicate_columns(train_df)

In [174]:
duplicate_columns

{'X31': ['X35', 'X37'],
 'X33': ['X39'],
 'X54': ['X76'],
 'X71': ['X84', 'X244'],
 'X11': ['X93',
  'X107',
  'X233',
  'X235',
  'X268',
  'X289',
  'X290',
  'X293',
  'X297',
  'X330',
  'X347'],
 'X90': ['X94', 'X242'],
 'X53': ['X102', 'X214', 'X239'],
 'X48': ['X113', 'X134', 'X147', 'X222'],
 'X118': ['X119'],
 'X88': ['X122', 'X243', 'X320'],
 'X138': ['X146'],
 'X62': ['X172', 'X216'],
 'X112': ['X199'],
 'X67': ['X213'],
 'X152': ['X226', 'X326'],
 'X125': ['X227'],
 'X29': ['X232', 'X279'],
 'X89': ['X245'],
 'X202': ['X247'],
 'X60': ['X248', 'X253', 'X385'],
 'X230': ['X254'],
 'X184': ['X262', 'X266'],
 'X295': ['X296'],
 'X298': ['X299'],
 'X44': ['X302'],
 'X58': ['X324'],
 'X155': ['X360'],
 'X240': ['X364', 'X365'],
 'X17': ['X382']}

In [175]:
for one_list in duplicate_columns.values():
    train_df.drop(columns=one_list, inplace=True)
    test_df.drop(columns=one_list, inplace=True)

In [176]:
print("after dropping")
print(train_df.shape)

after dropping
(4209, 322)


In [177]:
# finding correlated columns
corr_matrix = train_df.corr()

/tmp/ipykernel_11795/3555585889.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = train_df.corr()


In [178]:
# Get the column names of the DataFrame
columns = corr_matrix.columns

# Create an empty list to keep track of columns to drop
columns_to_drop = []

# Loop over the columns
for i in range(len(columns)):
    for j in range(i + 1, len(columns)):
        # Access the cell of the DataFrame
        if corr_matrix.loc[columns[i], columns[j]] > 0.95:
            columns_to_drop.append(columns[j])

print(len(columns_to_drop))

33


In [179]:
try:
    for one_list in columns_to_drop:
        train_df.drop(columns=one_list, inplace=True)
        test_df.drop(columns=one_list, inplace=True)
except Exception as ex:
    print(ex)

"['X162'] not found in axis"


In [180]:
train_df.shape

(4209, 315)

In [181]:
test_df.shape

(4209, 315)

In [182]:
train_df.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X374,X375,X376,X377,X378,X379,X380,X383,X384,y
0,0,k,v,at,a,d,u,j,o,0,...,0,0,0,1,0,0,0,0,0,130.81
1,6,k,t,av,e,d,y,l,o,0,...,0,1,0,0,0,0,0,0,0,88.53
2,7,az,w,n,c,d,x,j,x,0,...,0,0,0,0,0,0,0,0,0,76.26
3,9,az,t,n,f,d,x,l,e,0,...,0,0,0,0,0,0,0,0,0,80.62
4,13,az,v,n,f,d,h,d,n,0,...,0,0,0,0,0,0,0,0,0,78.02


In [183]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [184]:
def regression_results(actual_y, pred_y, X):
    '''This function takes actual and predicted value of target transform tem back and return scores of evaluation metrics'''

    # Reverse transforming the predicted output
    # y_true = np.expm1(actual_y)
    # y_pred = np.expm1(pred_y)
    y_true = actual_y
    y_pred = pred_y

    # Calculating regression metrics

    MAE = mean_absolute_error(y_true, y_pred)

    MSE = mean_squared_error(y_true, y_pred)

    RMSE = np.sqrt(MSE)

    r2 = r2_score(y_true, y_pred)

    adj_r2 = 1-(1-r2)*((X.shape[0]-1)/(X.shape[0]-X.shape[1]-1))

    return (round(MAE, 2), round(MSE, 2), round(RMSE, 2), round(r2, 2), round(adj_r2, 2))

In [185]:
def performance(model, X_train, X_test, y_train, y_test):
    '''This function takes model as input and return different evaluation metrics' score as dataframe'''

    # Doing prediction
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Calling the function regression_results
    train = regression_results(y_train, y_train_pred, X_train)
    test = regression_results(y_test, y_test_pred, X_test)

    # Storing the scores
    score = {'Metric': ['MAE', 'MSE', 'RMSE', 'r2', "adj_r2"],
             'Train Score': [train[0], train[1], train[2], train[3], train[4]],
             "Test Score": [test[0], test[1], test[2], test[3], test[4]]}

    # Create DataFrame
    df = pd.DataFrame(score)
    df.name = model
    return (df)

In [186]:
def feature_importance_linear(model, independent_var):
    '''This function takes instance of linear model as input and and return feature importance graph '''
    # Store the coefficients of the model into a dataframe
    coefs = pd.DataFrame(
        model.coef_,
        columns=['Coefficients'], index=independent_var)
    # Plot importance graph
    coefs.plot(kind='barh', figsize=(9, 7))
    plt.title('Model')
    plt.axvline(x=0, color='.5')
    plt.subplots_adjust(left=.3)

    plt.show()

In [187]:
def feature_importance_ensemble(model, independent_var):
    '''This function takes instance of ensemble model as input and returns the feature importance'''
    # Listing features
    features = independent_var
    # storing feature importance
    importances = model.feature_importances_
    indices = np.argsort(importances)
    # Plot the feature importance
    plt.title('Feature Importance')
    plt.barh(range(len(indices)),
             importances[indices], color='red', align='center')
    plt.yticks(range(len(indices)), [features[i] for i in indices])
    plt.xlabel('Relative Importance')

    plt.show()

### Splitting data into train and test

In [188]:
# checking for null values
train_df.isnull().sum()[train_df.isnull().sum() > 0]

Series([], dtype: int64)

In [189]:
train_df[cat_variables]

,X0,X1,X2,X3,X4,X5,X6,X8
0,k,v,at,a,d,u,j,o
1,k,t,av,e,d,y,l,o
2,az,w,n,c,d,x,j,x
3,az,t,n,f,d,x,l,e
4,az,v,n,f,d,h,d,n
...,...,...,...,...,...,...,...,...
4204,ak,s,as,c,d,aa,d,q
4205,j,o,t,d,d,aa,h,h
4206,ak,v,r,a,d,aa,g,e
4207,al,r,e,f,d,aa,l,u


In [190]:
train_df = pd.get_dummies(data=train_df, drop_first=True)

In [191]:
test_df = pd.get_dummies(data=test_df, drop_first=True)

In [192]:
X = train_df.drop(["y", "ID"], axis=1)
y = train_df["y"].values

In [193]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

# Random Forest Regressor

In [194]:

from sklearn.ensemble import RandomForestRegressor

In [195]:
print(X_train.shape)
X_train.head()

(3367, 492)


,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,...,X8_p,X8_q,X8_r,X8_s,X8_t,X8_u,X8_v,X8_w,X8_x,X8_y
3540,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3748,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1287,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2856,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1380,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [196]:
y_train

array([101.9 , 114.08, 106.31, ..., 102.77, 113.94, 110.1 ])

In [197]:
print(X_test.shape)
X_test.head()

(842, 492)


,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,...,X8_p,X8_q,X8_r,X8_s,X8_t,X8_u,X8_v,X8_w,X8_x,X8_y
3431,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2131,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2680,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
195,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3032,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [198]:
# RandomForest Regressor
model_rf = RandomForestRegressor()
model_rf.fit(X_train, y_train)
y_pred = model_rf.predict(X_test)

In [199]:
print("Scores of Random Forest")
performance(model_rf, X_train, X_test, y_train, y_test)

Scores of Random Forest


/tmp/ipykernel_11795/3540170249.py:19: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.name = model


,Metric,Train Score,Test Score
0,MAE,2.28,5.82
1,MSE,12.70,104.74
2,RMSE,3.56,10.23
3,r2,0.92,0.43
4,adj_r2,0.90,-0.36


In [200]:
from sklearn.ensemble import RandomForestRegressor


# Extract feature importance scores for label encoded data
fi_df2 = pd.DataFrame({
    'feature': X.columns,
    'rf_importance': model_rf.feature_importances_
}).sort_values(by='rf_importance', ascending=False)

fi_df2

,feature,rf_importance
246,X314,0.396049
247,X315,0.066176
95,X118,0.056117
18,X29,0.025584
206,X263,0.022234
...,...,...
456,X5_y,0.000000
332,X0_g,0.000000
307,X0_ac,0.000000
170,X207,0.000000


In [201]:
selected_50_columns = fi_df2.iloc[:50, :]

In [202]:
X = X[selected_50_columns["feature"].values]

In [203]:
id_columns = test_df["ID"]

In [204]:
test_df = test_df[selected_50_columns["feature"].values]

In [205]:
# train_test_split with new selected colum
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.2, random_state=0)

In [206]:
# RandomForest Regressor
model_rf = RandomForestRegressor()
model_rf.fit(X_train, y_train)
y_pred = model_rf.predict(X_test)

In [207]:
print("Scores of Random Forest")
performance(model_rf, X_train, X_test, y_train, y_test)

Scores of Random Forest


/tmp/ipykernel_11795/3540170249.py:19: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.name = model


,Metric,Train Score,Test Score
0,MAE,2.28,5.89
1,MSE,12.20,85.88
2,RMSE,3.49,9.27
3,r2,0.92,0.47
4,adj_r2,0.91,0.47


### Trying to predict test dataset

In [208]:
test_pred = model_rf.predict(test_df)

In [209]:
test_df["y"] = test_pred

In [211]:
test_df["ID"] = id_columns

In [212]:
predicted_df = test_df[["ID", "y"]]

In [ ]:
predicted_df

,ID,y
0,1,77.5206
1,2,91.9538
2,3,78.4345
3,4,78.2195
4,5,110.1608
...,...,...
4204,8410,107.4476
4205,8411,92.2216
4206,8413,89.2165
4207,8414,109.1032


In [ ]:
predicted_df.to_csv("predicted_results.csv")